In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My Drive/Colab Notebooks/acv assignments/assignment 2 modified/YOLO

/content/drive/My Drive/Colab Notebooks/acv assignments/assignment 2 modified/YOLO


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
!git clone https://github.com/experiencor/keras-yolo3
!mv keras-yolo3/* .
!rm -r keras-yolo3

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 220, done.
remote: Total 220 (delta 0), reused 0 (delta 0), pack-reused 220
Receiving objects: 100% (220/220), 91.26 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [0]:
import os
import numpy as np
import json
from voc import parse_voc_annotation
from yolo import create_yolov3_model, dummy_loss
from generator import BatchGenerator
from utils.utils import normalize, evaluate, makedirs
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from callbacks import CustomModelCheckpoint, CustomTensorBoard
from utils.multi_gpu_model import multi_gpu_model
import tensorflow as tf
import keras
from keras.models import load_model
from keras.callbacks import CSVLogger

Using TensorFlow backend.


In [0]:
!mkdir data
!mkdir data/yolo_val
!mkdir data/yolo_val/images
!mkdir data/yolo_val/annots

In [0]:
!ls

callbacks.py		  README.md
config.json		  requirements.txt
create_validation_set.sh  train.py
data			  utils
evaluate.py		  voc.py
gen_anchors.py		  YOLO
generator.py		  yolo3_one_file_to_detect_them_all.py
LICENSE			  yolo.py
predict.py		  zoo
__pycache__


In [0]:
!bash ./create_validation_set.sh data/images data/Annotations 1700 data/yolo_val

moving files 3954.jpg 3954.xml
moving files 4405.txt 4405.txt.xml
mv: cannot stat 'data/Annotations/4405.txt.xml': No such file or directory
moving files 1993.jpg 1993.xml
moving files 3071.txt 3071.txt.xml
mv: cannot stat 'data/Annotations/3071.txt.xml': No such file or directory
moving files 1412.jpg 1412.xml
moving files 1011.txt 1011.txt.xml
mv: cannot stat 'data/Annotations/1011.txt.xml': No such file or directory
moving files 0570.jpg 0570.xml
moving files 0261.jpg 0261.xml
moving files 4493.txt 4493.txt.xml
mv: cannot stat 'data/Annotations/4493.txt.xml': No such file or directory
moving files 0776.jpg 0776.xml
moving files 0881.txt 0881.txt.xml
mv: cannot stat 'data/Annotations/0881.txt.xml': No such file or directory
moving files 3815.jpg 3815.xml
moving files 4499.jpg 4499.xml
mv: cannot stat 'data/Annotations/4499.xml': No such file or directory
moving files 0776.txt 0776.txt.xml
mv: cannot stat 'data/Annotations/0776.txt.xml': No such file or directory
moving files 1415.jpg

In [0]:
!gdown https://drive.google.com/uc?id=1o_PM_zB6FxJRdLpEx8lEAhL29IMe1KzI

Downloading...
From: https://drive.google.com/uc?id=1o_PM_zB6FxJRdLpEx8lEAhL29IMe1KzI
To: /content/drive/My Drive/Colab Notebooks/acv assignments/assignment 2 modified/YOLO/backend.h5
249MB [00:03, 80.9MB/s]


In [0]:
config_json = """
{
    "model" : {
        "min_input_size":       288,
        "max_input_size":       448,
        "anchors":              [16,21, 20,45, 26,76, 33,28, 38,49, 59,33, 68,81, 99,147, 214,272],
        "labels":               ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14"]
    },

    "train": {
        "train_image_folder":   "data/images/",
        "train_annot_folder":   "data/Annotations/",
        "cache_name":           "sims_train.pkl",

        "train_times":          1,
        "batch_size":           1,
        "learning_rate":        1e-5,
        "nb_epochs":            1,
        "warmup_epochs":        0,
        "ignore_thresh":        0.5,
        "gpus":                 "0",

        "grid_scales":          [1,1,1],
        "obj_scale":            5,
        "noobj_scale":          1,
        "xywh_scale":           1,
        "class_scale":          1,

        "tensorboard_dir":      "logs",
        "saved_weights_name":   "sims.h5",
        "debug":                true
    },

    "valid": {
        "valid_image_folder":   "data/yolo_val/images/",
        "valid_annot_folder":   "data/yolo_val/Annotations/",
        "cache_name":           "sims_val.pkl",

        "valid_times":          1
    }
}
"""
with open('config.json', 'w') as f:
  f.write(config_json)

In [0]:
!python gen_anchors.py -c config.json

Traceback (most recent call last):
  File "gen_anchors.py", line 132, in <module>
    _main_(args)
  File "gen_anchors.py", line 111, in _main_
    centroids = run_kmeans(annotation_dims, num_anchors)
  File "gen_anchors.py", line 57, in run_kmeans
    indices = [random.randrange(ann_dims.shape[0]) for i in range(anchor_num)]
  File "gen_anchors.py", line 57, in <listcomp>
    indices = [random.randrange(ann_dims.shape[0]) for i in range(anchor_num)]
  File "/usr/lib/python3.6/random.py", line 189, in randrange
    raise ValueError("empty range for randrange()")
ValueError: empty range for randrange()


In [0]:
!python train.py -c config.json

Using TensorFlow backend.
2020-05-21 10:57:43.652318: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-05-21 10:57:43.652800: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14c2a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-21 10:57:43.652854: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-05-21 10:57:43.720950: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-21 10:57:43.772687: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-05-21 10:57:43.772766: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ea21a91ee18d): /proc/driver/nvidia/version does not exist

Seen labels: 	{'van': 1924, 'car': 

In [0]:
d = {"bus": 1286, 'car': 13747, 'van': 3735, 'truck': 1755, 'other': 426, 'airliner': 666, 'stairtruck': 299, 'pushbacktruck': 122, 'longvehicle': 1034, 'propeller': 123, 'chartered': 412, 'helicopter': 36, 'trainer': 403, 'boat': 6051, 'fighter': 35, 'pushback': 15, 'others': 119}
print(sorted(list(d.keys())))

['airliner', 'boat', 'bus', 'car', 'chartered', 'fighter', 'helicopter', 'longvehicle', 'other', 'others', 'propeller', 'pushback', 'pushbacktruck', 'stairtruck', 'trainer', 'truck', 'van']


In [0]:
import cv2

In [0]:
i = cv2.imread('data/images/1314.jpg')

In [0]:
i

array([[[ 34,  31,  33],
        [ 21,  18,  20],
        [ 20,  17,  19],
        ...,
        [147, 143, 142],
        [ 83,  77,  82],
        [ 53,  46,  53]],

       [[ 29,  26,  28],
        [ 13,  10,  12],
        [  8,   5,   7],
        ...,
        [ 78,  74,  73],
        [ 33,  27,  32],
        [ 24,  17,  24]],

       [[ 20,  17,  19],
        [ 11,   8,  10],
        [  5,   2,   4],
        ...,
        [ 28,  23,  20],
        [  5,   0,   1],
        [ 10,   3,   8]],

       ...,

       [[ 54,  63,  53],
        [ 56,  65,  55],
        [ 58,  66,  59],
        ...,
        [105, 104, 108],
        [112, 111, 113],
        [ 91,  90,  92]],

       [[ 54,  63,  53],
        [ 55,  64,  54],
        [ 56,  64,  57],
        ...,
        [ 92,  91,  95],
        [135, 134, 138],
        [150, 149, 153]],

       [[ 56,  65,  55],
        [ 56,  65,  55],
        [ 56,  64,  57],
        ...,
        [ 82,  81,  85],
        [139, 138, 142],
        [159, 158, 162]]